In [1]:
from FundData import *
from helper import Helper
import pprint, json, math
import fitz
import pandas as pd
from collections import defaultdict

#dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
#fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"

dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"

dry_path = r'\output\DryRun.pdf'
fin_path = r'\files\financial_indices.xlsx'
fund_path = r'C:\Users\rando\OneDrive\Documents\Dec 24'
json_path = dir_path + r'\output\dump.json'

mutual_fund = Helper.get_fund_paths(fund_path)

In [ ]:
""" SAMCO PDF FILE MAIN CODE"""

object = Samco(dir_path, dry_path, fin_path)
file_path = mutual_fund['Samco Mutual Fund']


fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']
bbox = object.PARAMS['clip_bbox']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
sample = input("Do you want to proceed (Y/N)?: ")

pages =  [3, 5, 7, 9, 11, 13, 15, 17, 18]

data = object.get_clipped_data(file_path,pages,bbox, fund_titles)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

#CREATE FINAL CONTENT    
# final_text = dict()
# for fund, items in extracted_text.items():
    
#     content_dict = dict()
#     for header, content in items.items():
#         header_content = object.match_regex_to_content(header, content)
#         content_dict.update(header_content)

#     final_text[fund] = content_dict

In [2]:
""" TATA FILE MAIN CODE """

object = Tata(dir_path, dry_path, fin_path)
file_path = mutual_fund["Tata Mutual Fund"]


fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']
bbox = object.PARAMS['clip_bbox']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
# sample = input("Do you want to proceed (Y/N)?: ")




Doc saved at: C:\Users\rando\OneDrive\Documents\mywork-repo\files\pdf_report.xlsx

Pages to extract: [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]


In [3]:
pages =  [i for i in range(17,84)]

data = object.get_clipped_data(file_path,pages,bbox, fund_titles)
data = object.extract_span_data(data,[])
# clean_data = object.process_text_data(data, data_cond) 
# nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
# extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [24]:
data_cond

[[5, 8], -15570765, 20.0, ['Swiss721BT-BoldCondensed']]

In [41]:
def process_text_data(text_data: dict, data_conditions: list):
        remove_text = ['Purchase', 'Amount', 'thereafter', '.', '. ', ',', ':', 'st', ';', "-", 'st ', ' ', 'th', 'th ', 'rd', 'rd ', 'nd', 'nd ', '', '`', '(Date of Allotment)']
        
        updated_text_data = {}

        for fund, data in text_data.items():
            blocks = data

            # Clean blocks
            cleaned_blocks = []
            for block in blocks:
                size, text, color, origin, bbox,font = block
                if text not in remove_text:
                    cleaned_blocks.append(block)

            # Process blocks (adjust size based on conditions)
            processed_blocks = []
            for block in cleaned_blocks:
                size, text, color, origin, bbox, font = block
                text = text.strip()
                
                if size in range(data_conditions[0][0], data_conditions[0][1]) and color == data_conditions[1] and font in data_conditions[3]:
                    size = data_conditions[2]  # Update size
                processed_blocks.append([size, text, color, origin, bbox,font])

            # # Group blocks by rounded y-coordinate
            # grouped_blocks = defaultdict(list)
            # for block in processed_blocks:
            #     y_coord = math.ceil(block[3][1])  # Extract and round the y-coordinate
            #     size = block[0]
            #     grouped_blocks[(y_coord, size)].append(block)

            # # Combine blocks with the same y-coordinate
            # combined_blocks = []
            # for key, group in grouped_blocks.items():
            #     if key[1] == data_conditions[2]:
            #         combined_text = " ".join(item[1] for item in group).strip()
            #         if combined_text:  # Ignore whitespace-only text
            #             size, color, origin, bbox,font = group[0][0], group[0][2], group[0][3], group[0][4], group[0][5]
            #             combined_blocks.append([size, combined_text, color, origin, bbox,font])
            #     else:
            #         for item in group:
            #             combined_blocks.append(item)

            updated_text_data[fund] = processed_blocks

        return updated_text_data

In [42]:
clean_data = process_text_data(data, [[5, 7], -15570765, 20.0, ['Swiss721BT-BoldCondensed']]) 
#nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)

In [43]:
clean_data

{'tata flexi cap fund': [[7,
   'As on 31st December 2024',
   -14475488,
   (47.13330078125, 69.326171875),
   (47.13330078125, 61.99017333984375, 134.41629028320312, 71.22317504882812),
   'OpenSans'],
  [20.0,
   'INVESTMENT STYLE',
   -15570765,
   (24.534700393676758, 88.05267333984375),
   (24.534700393676758,
    82.27467346191406,
    73.0027084350586,
    89.47467041015625),
   'Swiss721BT-BoldCondensed'],
  [5,
   'Primarily focuses on investing in equity and equity related',
   -14475488,
   (24.5, 96.51068115234375),
   (24.5, 91.6956787109375, 158.07699584960938, 97.69068145751953),
   'Swiss721BT-RomanCondense'],
  [5,
   'instruments of well researched companies across market',
   -14475488,
   (24.5, 102.51068115234375),
   (24.5, 97.6956787109375, 158.0950164794922, 103.69068145751953),
   'Swiss721BT-RomanCondense'],
  [5,
   'capitalization.',
   -14475488,
   (24.5, 108.51068115234375),
   (24.5, 103.6956787109375, 51.435001373291016, 109.69068145751953),
   'Swiss7

In [ ]:
""" FRANKLIN TEMPLETON FILE MAIN CODE"""

object = FranklinTempleton(dir_path, dry_path, fin_path)
file_path = mutual_fund['Franklin Templeton Mutual Fund']

fund_data = object.fund_data
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)

sample = input("Do you want to proceed (Y/N)?: ")
pages =  [i for i in range(17,51)]
bbox = object.content_bbox
data_cond = object.data_conditions

data = object.get_clipped_data(file_path,pages, bbox, fund_titles)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond)
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
""" BANDHAN MF FILE MAIN CODE"""

object = Bandhan(dir_path, dry_path, fin_path)
file_path = mutual_fund["Bandhan Mutual Fund"]

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
pages =  [i for i in range(17,83)]

data = object.extract_data_relative_line(file_path,pages,line_x ,'left', fund_titles) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
def quick_dump(data, path:str, indent=4):
        with open(path, "w") as file:
            json.dump(data, file, indent=indent)
            
quick_dump(extracted_text,json_path)       